In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime as dt
import json
import seaborn as sns
import re

In [2]:
members = pd.read_csv('../data/NSS_GreatCare_Members.csv', low_memory=False)
inpatient = pd.read_csv('../data/NSS_Hospital_Inpatient_Admissions.csv', low_memory=False)
pcp_visits = pd.read_csv('../data/NSS_PrimaryCare_Office_Visits.csv', low_memory=False)

In [ ]:
members.info()

In [ ]:
members1 = members[['Patient ID', 'Primary Plan ID']]

In [ ]:
members1.head()

In [21]:
inpatient.head()

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,1,1,21.0,NaN,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,1,1,21.0,348.4,Compression of brain,NaN,NaN,0,46465.40,250.0
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,1,1,21.0,NaN,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,1,1,21.0,NaN,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,2,1,NaN,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0


In [3]:
#running this datetime conversion before creating the new inpatient df.
inpatient['Encounter Start Date'] = pd.to_datetime(inpatient['Encounter Start Date'], format='%Y-%m-%d')
inpatient['Encounter End Date'] = pd.to_datetime(inpatient['Encounter End Date'], format='%Y-%m-%d')

In [ ]:
# Alternate, more efficient version of converting the dates
inpatient[['Encounter Start Date','Encounter End Date']] = inpatient[['Encounter Start Date','Encounter End Date']].apply(pd.to_datetime)

In [4]:
#Also need to convert this to datetime
pcp_visits['Encounter Date'] = pd.to_datetime(pcp_visits['Encounter Date'])

In [5]:
inpatient1=inpatient[['Encounter ID', 'Patient ID', 'Encounter Start Date', 'Encounter End Date']]

In [ ]:
inpatient1.info()

In [ ]:
pcp_visits.info()

In [ ]:
#to show all the rows:
#pd.set_option('display.max_columns', None)

# Question 1
### What is our 30-day all cause readmission rate across hospitals our members are admitted to?

Process:
1. if there is a second admission date, subtract the previous visit's end date .
2. Find where the time is <= 30 days

In [25]:
#first step is to sort the inpatient1 df 
inpatient1 = inpatient1.sort_values(['Patient ID',"Encounter Start Date", "Encounter End Date"], ascending = (False, True, True))

In [7]:
#THIS IS GOLD KEEP IT.  Creates columns for us to do math on and then identify rows where the patient has been admitted twice
inpatient1['Previous End Date'] = (inpatient1.groupby('Patient ID')['Encounter End Date'].shift(1))
inpatient1['Days Since Last Visit'] = inpatient1['Encounter Start Date'] - inpatient1['Previous End Date']
inpatient1 = inpatient1[inpatient1['Days Since Last Visit'].notnull()]




In [67]:
inpatient1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1512 entries, 6211 to 7
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   Encounter ID           1512 non-null   object         
 1   Patient ID             1512 non-null   object         
 2   Encounter Start Date   1512 non-null   datetime64[ns] 
 3   Encounter End Date     1512 non-null   datetime64[ns] 
 4   Previous End Date      1512 non-null   datetime64[ns] 
 5   Days Since Last Visit  1512 non-null   timedelta64[ns]
dtypes: datetime64[ns](3), object(2), timedelta64[ns](1)
memory usage: 82.7+ KB


In [8]:
#Creates new dataframe where time since last visit is less than or equal to 30 days
inpatient30d = inpatient1[inpatient1['Days Since Last Visit'] <= '30 days']

inpatient30d.info()
#755 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 755 entries, 6211 to 44
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype          
---  ------                 --------------  -----          
 0   Encounter ID           755 non-null    object         
 1   Patient ID             755 non-null    object         
 2   Encounter Start Date   755 non-null    datetime64[ns] 
 3   Encounter End Date     755 non-null    datetime64[ns] 
 4   Previous End Date      755 non-null    datetime64[ns] 
 5   Days Since Last Visit  755 non-null    timedelta64[ns]
dtypes: datetime64[ns](3), object(2), timedelta64[ns](1)
memory usage: 41.3+ KB


In [9]:
#Getting the readmission within 30 days rate:
inpatient['Patient ID'].nunique() #4701
inpatient30d['Patient ID'].nunique() #479

479

In [10]:
#doing the calculation
rate = inpatient30d['Patient ID'].nunique() / inpatient['Patient ID'].nunique()
print(rate) #10.18932%

0.10189321420974261


In [ ]:
479/4701

Question 1 answer is 10.18932% rate of radmission to hospital within 30 days of discharge

## Question 1 another way

In [11]:
#give the df a new name
inviz = inpatient1
#filter out null readmissions
inviz = inviz[inviz['Days Since Last Visit'].notnull()]
#make sure the readmissions are within 30 days
inviz = inviz[inviz['Days Since Last Visit'] <= '30 days']
inviz = inviz[inviz['Days Since Last Visit'] >= '0 days']

#and then how many people was it
total_readmission = len(inviz)
total_readmission
#this is a different count of readmissions compared to above.  Might be the >= to 0

703

In [12]:
len(inpatient)

6213

In [13]:
#using python to get the readmission rate/percentage
total_inpatients = len(inpatient)
readmission_rate = round(total_readmission/total_inpatients * 100, 2)
readmission_rate

11.31

In [ ]:
703/6213

This route gives 11.31%. 
Different from first time answering this question..

# Question2:
### What percent of patients have a PCP visit within a week of discharge?

In [65]:
#drop duplicates! on two columns
#two_top = two_top.sort_values(['Patient ID'], ascending = (False)).drop_duplicates(['Patient ID','Encounter End Date'])

#From Brenda:
#make df and merge
ad_cut = inpatient[['Patient ID','Encounter End Date']]
pcp_cut = pcp_visits[['Patient ID','Encounter Date']]
together = ad_cut.merge(pcp_cut, left_on ='Patient ID', right_on = 'Patient ID',how='left', indicator = False)
#sort
together = together.sort_values(['Patient ID',"Encounter Date", "Encounter End Date"], ascending = (False, True, True))
#change to datetime
together['Encounter Date'] = pd.to_datetime(together['Encounter Date'], format='%Y-%m-%d')
#sub encounter date from end dates
together['Days Since Last Visit'] = together['Encounter Date'] - together['Encounter End Date']
two_top = together[together['Days Since Last Visit'] <= '7 days']
two_top = two_top[two_top['Days Since Last Visit'] >= '0 days']
two_top.rename(columns={"Encounter End Date": "Inpatient End Date"}, inplace = True)
two_top

,Patient ID,Inpatient End Date,Encounter Date,Days Since Last Visit
44074,DS1|9981,2014-08-10,2014-08-11,1 days
44077,DS1|9981,2014-08-10,2014-08-12,2 days
44071,DS1|9979,2015-02-11,2015-02-15,4 days
44067,DS1|9978,2015-09-05,2015-09-11,6 days
44051,DS1|9945,2013-02-11,2013-02-14,3 days
...,...,...,...,...
238,DS1|10303,2012-06-24,2012-06-28,4 days
231,DS1|10303,2012-06-24,2012-06-30,6 days
50,DS1|10101,2012-12-09,2012-12-12,3 days
28,DS1|10073,2012-06-21,2012-06-27,6 days


Teng below

Process:
1. Identify patients who visited PCP within 7 days of hospital discharge
2. Identify total number of inpatient visits
3. divide answer to #1 by answer to #2

In [14]:
#Getting the relevant columns
x = inpatient[['Encounter ID','Patient ID', 'Encounter Start Date', 'Encounter End Date']]
y = pcp_visits[['Encounter ID','Patient ID', 'Encounter Date']]


In [15]:
#merge! 
inpatient_pcp= x.merge(y, how = 'left', on = 'Patient ID')
inpatient_pcp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 44098 entries, 0 to 44097
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Encounter ID_x        44098 non-null  object        
 1   Patient ID            44098 non-null  object        
 2   Encounter Start Date  44098 non-null  datetime64[ns]
 3   Encounter End Date    44098 non-null  datetime64[ns]
 4   Encounter ID_y        43791 non-null  object        
 5   Encounter Date        43791 non-null  datetime64[ns]
dtypes: datetime64[ns](3), object(3)
memory usage: 2.4+ MB


In [16]:
#Time between leaving hospital and visiting PCP as a new column

inpatient_pcp['7_Days'] = inpatient_pcp['Encounter Date'] - inpatient_pcp['Encounter End Date']

In [17]:
#Who visited their PCP within 7 days

# Looping through the time difference column to find any that are <= 7days and >= 0 days. 

t = []

for i in inpatient_pcp['7_Days']:
    if (i <= dt.timedelta(days=7)) and (i >= dt.timedelta(days=0)) :
        t.append('yes')
    else:
        t.append('no')

In [18]:
# Adding the list 't' as a new column 

inpatient_pcp['within_7'] = t

inpatient_pcp

,Encounter ID_x,Patient ID,Encounter Start Date,Encounter End Date,Encounter ID_y,Encounter Date,7_Days,within_7
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1228384|1043427248|1|SHIFTED,2015-12-08,1058 days,no
1,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|1326684|1043427248|1|SHIFTED,2012-06-18,-210 days,no
2,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|155317|1043427248|1|SHIFTED,2014-06-10,512 days,no
3,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|439093|1043427248|1|SHIFTED,2014-09-28,622 days,no
4,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,2013-01-12,2013-01-14,COV|PBSL|DS1|10033|P|711310|1043427248|1|SHIFTED,2014-03-16,426 days,no
...,...,...,...,...,...,...,...,...
44093,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|1238517|1407058787|2,2015-12-05,8 days,no
44094,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|302749|1407058787|2,2015-12-13,16 days,no
44095,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|781849|1205884392|1,2014-05-18,-558 days,no
44096,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,2015-11-24,2015-11-27,COV|PCSL|DS1|9994|P|93182|1043427248|1,2013-07-20,-860 days,no


In [19]:
#Get the counts of within_7 column
inpatient_pcp['within_7'].value_counts()

no     41723
yes     2375
Name: within_7, dtype: int64

### Dealing with duplicates (patients who saw their PCP more than once within the 7 days after discharge)

In [ ]:
#focus on the 'yes' from the 'within_7' column
yes = inpatient_pcp.loc[inpatient_pcp['within_7'] == 'yes']
type(yes)

In [ ]:
#remove duplicates from the yes ('Encounter ID_x' is from the inpatient visits - just count that and not multiple PCP visits)
yes = yes.drop_duplicates(subset='Encounter ID_x', keep='first')
yes #1864 rows (visits to PCP within 7 days)

1864 patients visited their PCP within 7 days of hospital discharge.

In [ ]:
#how many total inpatient visits were there?
x.info()

In [ ]:
#1864 patients / 6213 inpatients
1864/6213

# Question 3 
Do either vary by individual or group PCP's?

# Playing Around

In [ ]:
------- gonna try and visualize something

In [ ]:
inpatient1.head()

In [ ]:
#break out new columns for year and month
inpatient1['year'] = pd.DatetimeIndex(inpatient1['Previous End Date']).year
inpatient1['month'] = pd.DatetimeIndex(inpatient1['Previous End Date']).month
inpatient1.head()

In [ ]:
#How many under-30-days readmissions do we get, by month
by_month = inpatient1.groupby(['year','month'], as_index=False).count()
by_month = by_month.sort_values(['year','month','Patient ID'], ascending = (True, True,False))

In [ ]:
by_month.info()

In [ ]:
mean = by_month['Encounter ID'].mean

In [ ]:
sns.set_theme(style="whitegrid")
ax1 = sns.barplot(x='month', y='Encounter ID',  data=by_month)
ax1.axhline(mean, color='r', linestyle='--', label="Mean")
_=ax1.legend()

# Getting a list of Frequent Flier patients

Frequent flier = someone who has been readmitted more than 3 times within a year/365 days

In [ ]:
#just sorting stuff by admission date
inpatient3=inpatient1.sort_values(["Encounter Start Date"], ascending = (True))

In [59]:
#getting counts of how many visit each patient has had - all time in this data set
vc=inpatient3['Patient ID'].value_counts()
vc

DS1|3812     16
DS1|26751    14
DS1|53241    12
DS1|10451    11
DS1|21926    10
             ..
DS1|50615     1
DS1|27962     1
DS1|17450     1
DS1|16113     1
DS1|29282     1
Name: Patient ID, Length: 870, dtype: int64

In [ ]:
#list of patients who have come more than 3 times
inpatient3['Patient ID'].value_counts().reset_index(name="count").query("count > 3")["index"] #75 patients

### need to figure out if repeat visits are within a year

#### Bringing in Brenda's code for easier meshing

In [70]:
#importing with Brenda's variables to repurpose "readmissions within 30 days" for 365 days
mem = pd.read_csv('../data/NSS_GreatCare_Members.csv')
adm = pd.read_csv('../data/NSS_Hospital_Inpatient_Admissions.csv')
pcp = pd.read_csv('../data/NSS_PrimaryCare_Office_Visits.csv', low_memory = False)

In [71]:
#convert to datetime
adm['Encounter Start Date'] = pd.to_datetime(adm['Encounter Start Date'], format='%Y-%m-%d')
adm['Encounter End Date'] = pd.to_datetime(adm['Encounter End Date'], format='%Y-%m-%d')
pcp['Encounter Date'] = pd.to_datetime(pcp['Encounter Date'], format='%Y-%m-%d')

In [72]:
#More brenda for reuse
#clean and transform dfs
#order by patient then start date
one = adm.sort_values(['Patient ID',"Encounter Start Date", "Encounter End Date"], ascending = (False, True, True))
#To calculate the days between admissions:
#shift previous end date to next row correct format and calculate
one['Previous End Date'] = (one.groupby('Patient ID')['Encounter End Date'].shift(1))
one['Previous End Date'] = pd.to_datetime(one['Previous End Date'], format='%Y-%m-%d')
one['Days Since Last Visit'] = one['Encounter Start Date'] - one['Previous End Date']

#ad_count is the total number of admissions
adm_count = len(one)
adm_count

6213

In [74]:
#filter for readmission within 365 days
one_top = one
one_top = one_top[one_top['Days Since Last Visit'].notnull()]
one_top = one_top[one_top['Days Since Last Visit'] <= '365 days']
one_top = one_top[one_top['Days Since Last Visit'] >= '0 days']

#read_count is the number of patients who were readmitted within 365 days of their prior adm
read_count = len(one_top)
read_count

1345

In [77]:
#This is the patients who were readmitted more than 3 times within a year
inpatient4=one_top['Patient ID'].value_counts().reset_index(name="count").query("count > 3")["index"]
inpatient4

0      DS1|3812
1     DS1|26751
2     DS1|53241
3     DS1|10451
4     DS1|24041
        ...    
68    DS1|17710
69    DS1|59707
70    DS1|48279
71    DS1|10734
72    DS1|11581
Name: index, Length: 73, dtype: object

In [101]:
#New dataframe for only the frequent flier patients
newdf = one_top[(one_top['Patient ID'].isin(inpatient4))]
pd.set_option('display.max_rows', None)
newdf

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,Previous End Date,Days Since Last Visit
5851,HIA|DS1|6506|DS1|NPI|1689772592|2,DS1|6506,DS1|1,DS1|5047W,DS1|NPI|1689772592,DS1|NPI|1811018245,1.811018e+09,1689772592,2014-12-29,2015-01-07,...,21.0,NaN,NaN,N39.0,"Urinary tract infection, site not specified",0,37316.51,0.00,2014-08-17,134 days
5852,HIA|DS1|6506|DS1|NPI|1689772592|3,DS1|6506,DS1|1,DS1|5047W,DS1|NPI|1689772592,NaN,NaN,1689772592,2015-03-18,2015-03-21,...,23.0,NaN,NaN,T82.868A,"Thrombosis due to vascular prosthetic devices,...",0,12700.00,0.00,2015-01-07,70 days
5853,HIA|DS1|6506|DS1|NPI|1689772592|4,DS1|6506,DS1|1,DS1|5047W,DS1|NPI|1689772592,DS1|NPI|1811262090,1.811262e+09,1689772592,2015-03-31,2015-04-06,...,21.0,NaN,NaN,I12.0,Hypertensive chronic kidney disease with stage...,0,36892.00,0.00,2015-03-21,10 days
5844,HIA|DS1|6506|DS1|NPI|1033298617|1,DS1|6506,DS1|1,DS1|5047W,DS1|NPI|1033298617,DS1|NPI|1952345886,1.952346e+09,1033298617,2015-07-30,2015-08-10,...,21.0,NaN,NaN,I25.10,Atherosclerotic heart disease of native corona...,0,0.00,0.00,2015-04-06,115 days
5845,HIA|DS1|6506|DS1|NPI|1033298617|2,DS1|6506,DS1|1,DS1|5047W,DS1|NPI|1033298617,DS1|NPI|1952345886,1.952346e+09,1033298617,2015-08-27,2015-08-30,...,NaN,NaN,NaN,N18.6,End stage renal disease,0,0.00,0.00,2015-08-10,17 days
5846,HIA|DS1|6506|DS1|NPI|1033298617|3,DS1|6506,DS1|1,DS1|5047W,DS1|NPI|1033298617,DS1|NPI|1952345886,1.952346e+09,1033298617,2015-09-02,2015-09-07,...,21.0,NaN,NaN,N18.6,End stage renal disease,0,0.00,0.00,2015-08-30,3 days
5847,HIA|DS1|6506|DS1|NPI|1033298617|4,DS1|6506,DS1|1,DS1|5047W,DS1|NPI|1033298617,DS1|NPI|1952345886,1.952346e+09,1033298617,2015-09-12,2015-09-26,...,NaN,NaN,NaN,I82.621,Acute embolism and thrombosis of deep veins of...,0,0.00,0.00,2015-09-07,5 days
5848,HIA|DS1|6506|DS1|NPI|1033298617|5,DS1|6506,DS1|1,DS1|5047W,DS1|NPI|1033298617,DS1|NPI|1952345886,1.952346e+09,1033298617,2015-10-11,2015-10-31,...,NaN,NaN,NaN,J96.91,"Respiratory failure, unspecified with hypoxia",0,0.00,0.00,2015-09-26,15 days
5849,HIA|DS1|6506|DS1|NPI|1033298617|6,DS1|6506,DS1|1,DS1|5047W,DS1|NPI|1033298617,DS1|NPI|1164487989,1.164488e+09,1033298617,2015-11-06,2015-11-22,...,NaN,NaN,NaN,I12.0,Hypertensive chronic kidney disease with stage...,0,0.00,0.00,2015-10-31,6 days
5792,HIA|DS1|6139|DS1|NPI|1710918545|2,DS1|6139,DS1|1,DS1|5047W,DS1|NPI|1710918545,DS1|NPI|1659353969,1.659354e+09,1710918545,2014-03-25,2014-03-28,...,23.0,NaN,NaN,T82.867A,"Thrombosis due to cardiac prosthetic devices, ...",0,32039.38,250.00,2014-03-17,8 days
